In [258]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings(action='ignore')

# import boto3,os
# import sagemaker 
# from sagemaker import get_execution_role
# from sagemaker.amazon.amazon_estimator import get_image_uri
# from sagemaker.predictor import csv_serializer

In [259]:
d1=pd.read_csv('NEW_BASE_WITH_CRM.csv')

In [260]:
# from sagemaker.amazon import amazon_estimator

In [261]:
#import data from S3
# role=get_execution_role()
# d1=pd.read_csv('s3://{}/{}'.format('care-prod-actuarial-sandbox', 'ria/Banca_Renewal_Model_Data/BANCA_NEW_BASE_SANTOSH_WITH_CRM.csv'))

In [262]:
d1['if_crm']=d1['crm_data']

In [263]:
d=d1.loc[:,[ 'CHDRNUM', 'Polic_Start_Date', 'Policy_End_Date',
       'Final_Renewal1', 'GWP', 'Cover_type',
        'Owner_Covered', 'No_of_exposure',
       'Product_Desc', 'CLNTAGE', 'RIDESC', 'NBCATG',
       'Loading_Status',
        'HHCKFLG', 'HHCK_Cancelled', 'HHCK_Outstanding',
       'CareClassic_Zone', 'HHCK_Paid_Closed', 'Any_Rider', 'Ageband',
       'HHCK_Partially_Paid', 'IPDCLAIM', 'Risk_Type', 'Final_SI',
       'HHCK_Rejected','Policy_Related_flg', 'Claim_Related_flg', 'Renewal_Related_flg',
       'Health_Checkup_flg',
       'Enquiry_flg', 'Request_flg', 'if_crm']]

In [264]:
d['Calender_Month'] = pd.to_datetime(d['Policy_End_Date']).dt.month

#### Feature Engineering

In [265]:
col=['HHCK_Cancelled','HHCK_Paid_Closed','HHCK_Rejected','HHCK_Partially_Paid','HHCK_Outstanding']
for i in col:
    d[i]=np.where(d[i]=='HHCK',1,0)

In [266]:
d['HHCK_FINAL'] = d.HHCK_Paid_Closed + d.HHCK_Partially_Paid + d.HHCK_Outstanding
d['HHCK_rejected'] = d.HHCK_Cancelled + d.HHCK_Rejected

d['HHCK_FINAL'] = np.where(d.HHCK_FINAL>=1,1,0)
d['HHCK_rejected'] = np.where(d.HHCK_rejected >=1, 1,0)

d['Owner_Covered'] = np.where(d.Owner_Covered=='Y',1,0)
d['NBCATG'] = np.where(d.NBCATG=='NEW BUSINESS',1,0)

d['siBin'] = np.where(d.Final_SI<500000,1,
                       np.where(d.Final_SI==500000,2,3))

In [267]:
d.drop(columns=['HHCK_Outstanding', 'HHCK_Cancelled', 'HHCK_Paid_Closed','HHCK_Rejected',
               'HHCK_Partially_Paid','Owner_Covered','Policy_End_Date'],axis=1,inplace=True)

In [268]:
d=d.dropna()

In [269]:
d.shape

(170923, 29)

In [270]:
ind=d[d['GWP']<=10].index

In [271]:
d.drop(ind,axis=0,inplace=True)

In [272]:
d[d['GWP']<=d['GWP'].quantile(q=0.03)].shape

(5156, 29)

In [273]:
d['GWP'].quantile(q=0.965)

25209.0

In [274]:
d['gwpBin']= np.where(d.GWP <= d['GWP'].quantile(q=0.02), 1,
                                np.where(d.GWP <=d['GWP'].quantile(q=0.25),2,
                                  np.where(d.GWP <=d['GWP'].quantile(q=0.5),3,  
                                np.where(d.GWP <=d['GWP'].quantile(q=0.75),4,
                                np.where(d.GWP <=d['GWP'].quantile(q=0.965),5 ,6
                                  )))))

In [275]:
d['gwpBin'].value_counts(normalize=True)*100

4    25.093322
3    24.975134
2    22.975883
5    21.360450
6     3.498836
1     2.096376
Name: gwpBin, dtype: float64

In [276]:
pd.crosstab(d['gwpBin'],d['Final_Renewal1'],margins=True,normalize='index')

Final_Renewal1,0,1
gwpBin,,
1,0.316215,0.683785
2,0.635794,0.364206
3,0.493979,0.506021
4,0.434877,0.565123
5,0.329900,0.670100
6,0.403846,0.596154
All,0.469804,0.530196


In [277]:
#function to change bins to RIDESC

Pvt_Bank=['INDUS BANK LTD','RBLK BANK LIMITED']
Psu_bank=['Royal National Bank Ltd','KINGDOM BANK']
global Pvt_Bank,Psu_bank
def fl(i):
    if i in Pvt_Bank:
        return 'Pvt_Bank'
    elif i in Psu_bank:
        return 'Psu_bank'
    else:
        return 'Others'

In [278]:
d['RIDESC']=d['RIDESC'].apply(fl)

In [279]:
#function to change bins in IPDClaim
def m(xf):
    if xf <=0:
        return 0
    else:
        return 1

In [280]:
d['IPDCLAIM']=d['IPDCLAIM'].apply(m)

In [281]:
#function to change bins in Product Desc
c=['Cust GR Care','Group Care 360']
f=['Retail ']
global c,d
def prm(x):
    if x in c:
        return 'Group Care 360'
    elif x in f:
        return 'Care'
    else:
        return 'Others'

In [282]:
d['Product_Desc']=d['Product_Desc'].apply(prm)

In [283]:
#function to change floatercombination
ci=['00 - 18 Years','18 - 24 Years']
pi=['51 - 55 Years','56 - 60 Years','46 - 50 Years']
di=['61 - 65 Years','66 - 70 Years','71 - 75 Years','Above 75 Years']
by=['36 - 40 Years','41 - 45 Years']
li=['25 - 35 Years']
global ci,di,pi,by,li
def fli(i):
    if i in ci:
        return 1
    elif i in pi:
        return 4
    elif i in di:
        return 5
    elif i in by:
        return 3
    elif i in li:
        return 2
    else:
        return 6

In [284]:
d['Ageband']=d['Ageband'].apply(fli)

In [285]:
d['Calender_Month'].replace(to_replace=[2,3,7,8,9,10,11],value=2,inplace=True)
d['Calender_Month'].replace(to_replace=[4,6],value=4,inplace=True)

Dropping columns due to heavy correlation

In [286]:
d.drop(columns=['Policy_Related_flg','Enquiry_flg','Request_flg'],inplace=True, axis=1)

In [287]:
d.shape

(170914, 27)

In [288]:
mod_df=d.copy()

In [289]:
df_cat=mod_df.select_dtypes(['object'])
df_num=mod_df.select_dtypes(['int64','float64'])

In [290]:
mod_df=pd.get_dummies(mod_df,columns=[ 'Product_Desc', 'RIDESC',
       'CareClassic_Zone','Cover_type'
       ],drop_first=True)

In [291]:
mod_df.drop(columns=['Polic_Start_Date','CHDRNUM','GWP', 'Final_SI',
        'Loading_Status','CLNTAGE', 'HHCKFLG','HHCK_rejected','Risk_Type'],inplace=True,axis=1)

In [292]:
mod_df.drop(columns=[
 'No_of_exposure',
 'Any_Rider','Cover_type_MF',
    'Claim_Related_flg',
 'Health_Checkup_flg'
    ],inplace=True,axis=1)

In [293]:
mod_df.shape

(170914, 17)

In [294]:
mod_df.columns

Index(['Final_Renewal1', 'NBCATG', 'Ageband', 'IPDCLAIM',
       'Renewal_Related_flg', 'if_crm', 'Calender_Month', 'HHCK_FINAL',
       'siBin', 'gwpBin', 'Product_Desc_Group Care 360', 'Product_Desc_Others',
       'RIDESC_Psu_bank', 'RIDESC_Pvt_Bank', 'CareClassic_Zone_Zone 2',
       'CareClassic_Zone_Zone 3', 'Cover_type_IN'],
      dtype='object')

# MODEL BUILDING

In [295]:
X=mod_df.drop('Final_Renewal1',axis=1)
Y1=mod_df.pop('Final_Renewal1')
Y=pd.DataFrame(Y1,columns=['Final_Renewal1'])

In [296]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y,train_size=0.80,random_state=31,shuffle=True)

In [297]:
from sklearn.ensemble import RandomForestClassifier
XGModel1=RandomForestClassifier(n_estimators=500)
XGModel1.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=500)

In [ ]:
#PUSHING MODEL ARTIFACTS TO S3
# import pickle
# bucket='care-prod-actuarial-sandbox'
# s3 = boto3.client('s3')
# prefix = "ria/Banca_Renewal_Model_Data/"+'Bancamodel.pkl'
# s3.put_object(Bucket=bucket, Key=prefix, Body=pickle.dumps(XGModel1))

**Model Evaluation**

In [298]:
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
print(round(XGModel1.score(X_train,Y_train),4))
print('Accuracy: ',round(XGModel1.score(X_test,Y_test),4))
print('***********************************')
print(roc_auc_score(Y_train,XGModel1.predict(X_train)))
print('ROC_AUC_Score: ',round(roc_auc_score(Y_test,XGModel1.predict(X_test)),4))
print('***********************************')
print(classification_report(Y_test,XGModel1.predict(X_test)))
# plot_confusion_matrix(XGModel1, X_test, Y_test,values_format = 'd')

0.7355
Accuracy:  0.6956
***********************************
0.7352071451103686
ROC_AUC_Score:  0.695
***********************************
              precision    recall  f1-score   support

           0       0.67      0.69      0.68     16069
           1       0.72      0.70      0.71     18114

    accuracy                           0.70     34183
   macro avg       0.69      0.69      0.69     34183
weighted avg       0.70      0.70      0.70     34183



**Decile Analysis**

In [299]:
#df with probabilities scores-

test_pred=pd.DataFrame(XGModel1.predict_proba(X_test),index=X_test.index, columns=['score0','score1'])

X_test1=X_test.copy()
df_=d.copy()

X_test1['index'] = X_test.index
df_['index'] = d.index

X_test1['train_test'] = 0

X_test1['score1'] = test_pred['score1']

d2_new = df_.merge(X_test1[['train_test','index','score1']],on='index',how='left')
d2_new1 = d2_new[d2_new.train_test==0]
d2_new1['Decile_rank'] = pd.qcut(d2_new1['score1'], 10,
                                       labels = False,duplicates='drop')

In [300]:
# create a list of our conditions
conditions = [
    (d2_new1['Decile_rank'] < 6),
    (d2_new1['Decile_rank'] >= 6) & (d2_new1['Decile_rank'] <= 7),
    (d2_new1['Decile_rank'] > 7)
    ]

# create a list of the values we want to assign for each condition
values = ['Worst_Renewal', 'Good_Renewal', 'Excellent_Renewal']

# create a new column and use np.select to assign values to it using our lists as arguments
d2_new1['Performance_Category'] = np.select(conditions, values)

# display updated DataFrame
# d2_new1.head()

In [ ]:
# pushing the results to S3
# d2_new1.to_csv('New_Base_withCRM_results.csv')

# prefix='ria/Banca_Renewal_Model_Data/'
# bucket='care-prod-actuarial-sandbox'
# boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'New_Base_withCRM_results.csv')).upload_file('New_Base_withCRM_results.csv')

In [ ]:
# import seaborn as sns
# # fig.set_size_inches(15,7)
# sns.scatterplot(x='score1',y='Decile_rank',data=d2_new1)
# plt.xticks(np.arange(0,1,0.1))
# plt.yticks(np.arange(0,10,1))
# plt.show()


# ct=pd.crosstab(d2_new1.Final_Renewal1,d2_new1.Decile_rank,margins=False).T
# ct

In [ ]:
# # abc=(XRModel1.coef_[0])
# abc=XGModel1.feature_importances_
# xyz=X_train.columns
# abc1 = pd.concat([pd.Series(abc),pd.Series(xyz)], axis=1)
# abc1.columns = ['importance', 'varname']
# abc1=abc1.sort_values(by='importance',ascending=False)
# abc1['%age']=abc1['importance'].cumsum()
# # xyz
# abc1

In [301]:
import pickle
pickle.dump(XGModel1, open('Renewal_Model_updated.pkl', 'wb'))